# ASR + LLM Fine-tuning with ESPnetEZ

This Jupyter notebook provides a step-by-step guide on using the ESPnetEZ trainer to fine-tune ASR + LLM. In this demonstration, we will leverage the `MuST-C-v2` dataset (English to German subset) to fine-tune this cascade Speech Translation (ST) system.

In this notebook, we assume that you have already downloaded the `MuST-C-v2` dataset and created the dump file using the recipe. If you haven't done this and are unfamiliar with the recipes provided in ESPnet, you can refer to the data preparation sections in the `train_from_scratch.ipynb` or `finetune_owsm.ipynb` notebooks in the ASR demos.

Author: Masao Someki [@Masao-Someki](https://github.com/Masao-Someki)

Let's install espnet and transformers if you haven't installed it.

In [ ]:
!pip install -U espnet
!pip install transformers

And import necessary libraries and set several hyperparameters.

In [ ]:
import torch
import torch.nn as nn
import numpy as np
import librosa
from pathlib import Path
from espnet2.layers.create_adapter_fn import create_lora_adapter
from espnet2.asr.espnet_model import ESPnetASRModel
from espnet2.train.dataset import kaldi_loader
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

import espnetez as ez

CONFIG = "owsm_finetune_base"
TRAIN_KEY = "huggingface_cascade"
FINETUNE_MODEL = "pyf98/librispeech_100_e_branchformer"
HF_MODEl = "google-t5/t5-base"

DATA_PATH = "./data"
DUMP_DIR = "./dump/raw"
EXP_DIR = f"./exp/train_{TRAIN_KEY}"
STATS_DIR = f"./exp/stats_{TRAIN_KEY}"


## Data Preparation

In this demo, let's define our custom dataset using the prepared dump files. This dataset will load the audio and text data from the dump files and convert the text data for our training.

In [ ]:

class CustomDataset:
    def __init__(self, data_path, is_train=True):
        self.data_path = data_path
        if is_train:
            data_path = f"{data_path}/train.en-de_sp"
        else:
            data_path = f"{data_path}/dev.en-de"
        
        self.data = {}
        with open(f"{data_path}/text.tc.de", "r") as f:
            for line in f.readlines():
                audio_id, translated = line.strip().split(maxsplit=1)
                translated = translated.replace(" &apos;", "'")\
                                       .replace(" &quot;", '"')\
                                       .replace(" &amp;", "&")
                self.data[audio_id] = {
                    'translated': translated
                }
        
        with open(f"{data_path}/text", "r") as f:
            for line in f.readlines():
                audio_id, text = line.strip().split(maxsplit=1)
                text = text.replace(" &apos;", "'")\
                           .replace(" &quot;", '"')\
                           .replace(" &amp;", "&")
                self.data[audio_id]['text'] = text
        
        self.keys = list(self.data.keys())
        self.loader = kaldi_loader(f"{data_path}/wav.scp")
    
    def __len__(self):
        return len(self.keys)
    
    def __getitem__(self, idx):
        # This output will be fed into the lambda function in `data_info`.
        idx = int(idx)
        return {
            'speech': self.loader[self.keys[idx]].astype(np.float32),
            'text': self.data[self.keys[idx]]['text'],
            'translated': self.data[self.keys[idx]]['translated']
        }


## Model Preparation

Next, let's prepare the `build_model_fn` function for the Trainer. We will define our custom model for ASR + LLM training. In this demo, we will train the ASR model with the ASR text, and the LLM will be fine-tuned with the translated text and ASR output.

We will use the `ESPnetASRModel` for training. Currently, we don't have a specific class to support custom models, so we will leverage the existing ASR class.

The `forward` method will take the output of the `data_info` and `_lengths` tensors, and output the loss for training. I have also added logging functionality to track the training progress inside the `forward` method.

In [ ]:

class CustomFinetuneModel(ESPnetASRModel):
    def __init__(self, model, log_every=500):
        super().__init__(
            vocab_size=model.vocab_size,
            token_list=model.token_list,
            frontend=model.frontend,
            specaug=model.specaug,
            normalize=model.normalize,
            preencoder=model.preencoder,
            encoder=model.encoder,
            postencoder=model.postencoder,
            decoder=model.decoder,
            ctc=model.ctc,
            ctc_weight=model.ctc_weight,
            interctc_weight=model.interctc_weight,
            ignore_id=model.ignore_id,
            joint_network=None,
            lsm_weight=0.0,
            length_normalized_loss=False,
            report_cer=False,
            report_wer=False,
            sym_space="<space>",
            sym_blank="<blank>",
            sym_sos = "<sos>",
            sym_eos = "<eos>",
            # sym_sop = "<sop>",  # start of prev
            # sym_na = "<na>",  # not available
            extract_feats_in_collect_stats=model.extract_feats_in_collect_stats,
        )
        self.iter_count = 0
        self.log_every = log_every
        self.log_stats = {
            'loss': 0.0,
            'acc': 0.0,
            'lm_loss': 0.0,
        }
        self.lm = AutoModelForSeq2SeqLM.from_pretrained(
            HF_MODEl,
            device_map = 'cuda'
        )
        self.lm_tokenizer = AutoTokenizer.from_pretrained(HF_MODEl)
    
    def get_text(self, token_ids):
        return [
            tokenizer.tokens2text(converter.ids2tokens(token_id))
            for token_id in token_ids
        ]
    
    def forward(
        self,
        speech: torch.Tensor,
        speech_lengths: torch.Tensor,
        text: torch.Tensor,
        text_lengths: torch.Tensor,
        text_spk2: torch.Tensor,
        text_spk2_lengths: torch.Tensor,
        **kwargs,
    ):
        # compute ASR loss
        loss, stats, weight = super().forward(
            speech, speech_lengths,
            text, text_lengths,
        )

        # compute hf loss
        target_texts = self.get_text(text_spk2)
        target_tokens = self.lm_tokenizer(target_texts, return_tensors="pt", padding=True).input_ids.to(speech.device)
        input_texts = self.get_text(text)
        input_tokens = self.lm_tokenizer(input_texts, return_tensors="pt", padding=True).input_ids.to(speech.device)
        lm_output = self.lm(input_ids=input_tokens, labels=target_tokens)

        # Add lm loss to ASR loss
        loss += lm_output.loss
        stats['lm_loss'] = lm_output.loss.detach()

        self.log_stats['loss'] += stats['loss'].item()
        self.log_stats['acc'] += stats['acc'].item()
        self.log_stats['lm_loss'] += lm_output.loss.item()

        self.iter_count += 1
        if self.iter_count % self.log_every == 0:
            _loss = self.log_stats['loss'] / self.log_every
            _acc = self.log_stats['acc'] / self.log_every
            lm_loss = self.log_stats['lm_loss'] / self.log_every
            print(f"[{self.iter_count}] - loss: {_loss:.3f} - acc: {_acc:.3f} - lm_loss: {lm_loss:.3f}")
            self.log_stats['loss'] = 0.0
            self.log_stats['acc'] = 0.0
            self.log_stats['lm_loss'] = 0.0

        return loss, stats, weight


Then let's define the `data_info` and `build_model_fn` functions for the Trainer.

In [ ]:
data_info = {
    "speech": lambda d : d['speech'],
    "text": lambda d : tokenize(d['text'].upper()),
    "text_spk2": lambda d : tokenize(d['translated'].upper()),
} 

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def build_model_fn(args):
    pretrained_model = Speech2Text.from_pretrained(
        FINETUNE_MODEL,
        beam_size=10,
    )
    model = pretrained_model.asr_model
    model.train()
    model = CustomFinetuneModel(model, log_every=20)
    return model


## Training

Finally, let's define the training configuration, instanciate the dataset and trainer and start the training!


In [ ]:
from espnet2.bin.asr_inference import Speech2Text
pretrained_model = Speech2Text.from_pretrained(
    FINETUNE_MODEL,
    beam_size=10,
    device="cpu"
)
tokenizer = pretrained_model.tokenizer
converter = pretrained_model.converter

training_config = vars(pretrained_model.asr_train_args)
del pretrained_model

finetune_config = ez.config.update_finetune_config(
    "asr",
    training_config,
    f"conf/{CONFIG}.yaml"
)
finetune_config['multiple_iterator'] = False

def tokenize(text):
    return np.array(converter.tokens2ids(tokenizer.text2tokens(text)))


In [ ]:
train_dataset = CustomDataset(data_path="./dump/raw", is_train=True)
dev_dataset = CustomDataset(data_path="./dump/raw", is_train=False)

train_dataset = ez.dataset.ESPnetEZDataset(train_dataset, data_info=data_info)
dev_dataset = ez.dataset.ESPnetEZDataset(dev_dataset, data_info=data_info)

In [ ]:
trainer = ez.Trainer(
    task="asr",
    train_config=finetune_config,
    train_dataset=train_dataset,
    valid_dataset=dev_dataset,
    data_info=data_info,
    build_model_fn=build_model_fn,
    output_dir=EXP_DIR,
    stats_dir=STATS_DIR,
    ngpu=1,
)
trainer.collect_stats()
trainer.train()